# Example 3: Grandma Portfolio Allocation

This example demostrates how to build a easily managable portfolio using the Grandma Stock Valuation model.
* Load Data
* Portfolio Allocation - the Basic Idea
* Build a Portfolio with SP500 ETF and Cash


In [1]:
from grandma_stock_valuation import FileLogger, loadPacakgeData
from grandma_stock_valuation import batchValuation, addCashPortfolio, getCorrelationWeight, allocatePortfolio

# Refer to example_0_FileLogger.ipynb for details of the FileLogger.
logger = FileLogger()
logPrint = logger.logPandas

### Load Data

For this example, I will use the stored package data.

To query data from Yahoo, please refer to *example_1_yahoo_data_loader.ipynb*.

In [2]:
d_instrument_data, d_instrument = loadPacakgeData(verbose=2)

d_instrument

EEMA data contains 2530 rows, 2530 dates from 2012-02-09 to 2022-02-28.
IEV data contains 5431 rows, 5431 dates from 2000-07-28 to 2022-02-28.
IVV data contains 5479 rows, 5479 dates from 2000-05-19 to 2022-02-28.
VPL data contains 4273 rows, 4273 dates from 2005-03-10 to 2022-02-28.


{'IVV': 'SP500',
 'VPL': 'Developed Asia-Pacific',
 'IEV': 'Europe',
 'EEMA': 'Emerging Asia'}

In [3]:
logPrint("Keys of d_instrument_data:", str(d_instrument_data.keys()))

logPrint("IVV (SP500 ETF):", d_instrument_data['IVV'].head())

2022-03-02 21:14:50,638 INFO Keys of d_instrument_data: dict_keys(['EEMA', 'IEV', 'IVV', 'VPL'])
2022-03-02 21:14:50,736 INFO IVV (SP500 ETF): 
        date       open       high        low      close  close_adj   volume
0 2000-05-19  142.65625  142.65625  140.25000  140.68750  94.121216   775500
1 2000-05-22  140.59375  140.59375  136.81250  139.81250  93.535789  1850600
2 2000-05-23  140.21875  140.21875  137.68750  137.68750  92.114151   373900
3 2000-05-24  137.75000  140.06250  136.65625  139.75000  93.494003   400300
4 2000-05-25  140.03125  140.93750  137.87500  138.46875  92.636810    69600



### Portfolio Allocation - the Basic Idea

*Should I buy or sell?*

Is this question daunting you all the time? Well, how about thinking about investment from a **portfolio management** perspective?

*What is it about?*

**Portfolio management** is both old and new to most retail investors.
* On one hand, most people have heard about it, and some may even have mathematical knowledge about the techniques.
* On the other hand, most people don't know how to put it into action, and some may even have little clue about what investment instruments they are holding!

Let's go straight into the points:
* How much CASH shall I invest into stocks? - `Grandma` will answer it.
* How many instruments should I hold? - `Grandma` will help you with it.
* How frequency do I need to adjust my portfolio? - monthly or longer.
* How Grandma Profolio Allocation works? - allocate more to more under-valued instruments.


### Build a Portfolio with SP500 ETF and Cash

Let's start with a simple portfolio consists of the SP500 ETF (IVV) and cash.

*What? Can I call this a portfolio?* - Of course it is!

Cell below get valuation of the SP500 ETF using Grandma Stock Valuation model - refer to *example_2_grandma_valuation.ipynb* for the details.

In [5]:
d_SP500 = {ticker:daily_price for ticker, daily_price in d_instrument_data.items() if ticker=='IVV'}

df_SP500_valuation, _ = batchValuation(d_SP500, save_result=False)

df_SP500_valuation

,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_years
0,IVV,0.971054,10.00274,0.134314,438.720001,411.907796,0.065093,0.48463


We can use the `addCashPortfolio()` function to add cash into the about valuation table. This function takes the following arguments:
* `df_valuation_metrics` (pandas.DataFrame): a dataframe with the valuation metrics of an existing portfolio.<br>It is usually the output of `batchValuation()` function.
* `id_col` (str): column in `df_valuation_metrics` as the identifier of the instruments in the portfolio.<br>Default to "ticker".
* `cash_name` (str): a name, default to "cash", to be appended under the `id_col`.
* `value_col` (str): column in `df_valuation_metrics` with the valuations of each instruments in the portfolio.<br>Default to `over_value_years`.
* `cash_value` (float): Valuation of cash. I will just use 0 (neither over-valued nor under-valued).<br>You may want to take inflation or other factor into consideration - up to your decision.



In [9]:
df_SP500_and_cash = addCashPortfolio(df_SP500_valuation, cash_value=0.0)

df_SP500_and_cash

,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_years
0,IVV,0.971054,10.00274,0.134314,438.720001,411.907796,0.065093,0.48463
1,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.00000


WIP

In [10]:
df_SP500_and_cash['allocation'] = allocatePortfolio(df_SP500_and_cash['over_value_years'], with_cash=True)

df_SP500_and_cash

,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_years,allocation
0,IVV,0.971054,10.00274,0.134314,438.720001,411.907796,0.065093,0.48463,0.381159
1,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.618841
